In [2]:
# -*- coding: utf-8 -*-
# INVENTORYAI - 5. hét: RAG integráció és kérdés-válasz modell (GPT-5)
# Notebook neve: inventoryAI_week5_RAG_integration.ipynb

# -------------------------------
# 1️⃣ Környezet és csomagok importálása
# -------------------------------
import os
from dotenv import load_dotenv
import pandas as pd
from qdrant_client import QdrantClient
from openai import OpenAI

# -------------------------------
# 2️⃣ .env betöltése és OpenAI kulcs ellenőrzése
# -------------------------------
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("❌ Nincs beállítva az OPENAI_API_KEY környezeti változó!")

client_openai = OpenAI(api_key=api_key)

# -------------------------------
# 3️⃣ Qdrant kliens inicializálása
# -------------------------------
qdrant_client = QdrantClient(url="http://localhost:6333")

# -------------------------------
# 4️⃣ Large modell és kollekció beállítása
# -------------------------------
model_name = "text-embedding-3-large"
collection_name = "leltar_text_embedding_3_large_all_columns"

# -------------------------------
# 5️⃣ Ellenőrizzük, hogy a kollekció létezik
# -------------------------------
if not qdrant_client.collection_exists(collection_name=collection_name):
    raise RuntimeError(f"❌ A kollekció '{collection_name}' nem található! Töltsd fel a 3. heti notebookból.")

print(f"✅ Kollekció készen áll: {collection_name}")

# -------------------------------
# 6️⃣ Felhasználói kérdés bekérése
# -------------------------------
# Konzolról vagy notebook cellából is adhatjuk
user_question = input("Írd be a kérdésed az inventoryhoz: ")

# -------------------------------
# 7️⃣ Kérdés embedding generálása a large modellel
# -------------------------------
response = client_openai.embeddings.create(
    model=model_name,
    input=user_question
)
query_embedding = response.data[0].embedding

# -------------------------------
# 8️⃣ Legrelevánsabb rekordok lekérése Qdrantból
# -------------------------------
search_results = qdrant_client.query_points(
    collection_name=collection_name,
    query=query_embedding,
    limit=5  # Top 5 találat
)

# -------------------------------
# 9️⃣ Kontextus előkészítése GPT-5 számára
# -------------------------------
context_lines = []
for hit in search_results.points:
    # Minden payload mezőt szövegesen összefűzünk
    line = " | ".join([f"{col}: {val}" for col, val in hit.payload.items()])
    context_lines.append(line)

context_text = "\n".join(context_lines)

# -------------------------------
# 🔟 GPT-5 hívás a válasz generálására
# -------------------------------
chat_response = client_openai.chat.completions.create(
    model="gpt-5",
    messages=[
        {"role": "system", "content": "Segítséget nyújtok inventory kérdésekre."},
        {"role": "user", "content": f"Kérdés: {user_question}\nReleváns adatok:\n{context_text}"}
    ]
)

# -------------------------------
# 1️⃣1️⃣ Kérdés kiírása
# -------------------------------
print("\n❓ Kérdés:")
print(user_question)

# -------------------------------
# 1️⃣2️⃣ AI válasz kiírása
# -------------------------------
answer = chat_response.choices[0].message.content
print("\n📝 AI válasz:")
print(answer)

✅ Kollekció készen áll: leltar_text_embedding_3_large_all_columns

❓ Kérdés:
van elérhető matlab szoftver?

📝 AI válasz:
Igen. Az állományban két MATLAB-hoz kapcsolódó tétel szerepel:

- Eszköz 1000113 – „szoftver matlab 25 felhasználós szh-1/08”
  - Leltárszám: 14376
  - Telephely: nikszf4421
- Eszköz 1000169 – „10 svite (matlab, simulink) szh-18/08”
  - Leltárszám: 17922
  - Telephely: nikami4410

Ha megmondod, melyik telephelyen és milyen célra kell, tudok pontosítani vagy jelezni a felelősök felé a hozzáférés/telepítés ügyében.


# InventoryAI – Week 5: Kérdés-válasz modell kipróbálása (GPT-5)

## 1. Környezet előkészítése

- Betöltjük a szükséges könyvtárakat (pl. OpenAI kliens, Qdrant kliens).
- Ellenőrizzük, hogy a **OpenAI API kulcs** be van-e állítva a `.env` fájlban.
- Inicializáljuk a **Qdrant klienst**, amely a vektoralapú kereséshez szükséges.

---

## 2. Kollekció ellenőrzése

- A 3. heti feladat során feltöltött **`leltar_text_embedding_3_large_all_columns`** kollekciót használjuk.
- Ellenőrizzük, hogy a kollekció létezik, mert innen kerülnek lekérésre a releváns rekordok.

---

## 3. Felhasználói kérdés bekérése

- A felhasználó beírja a kérdését az inventory adatokkal kapcsolatban.
- Például:
  `"Melyik telephelyen van a MATLAB szoftver?"`

---

## 4. Kérdés embedding generálása

- A felhasználó kérdését a **`text-embedding-3-large`** modell segítségével **numerikus vektorrá alakítjuk** (embedding).
- Ez a vektor reprezentálja a kérdés jelentését a magas dimenziós térben.
- Ez a lépés szükséges ahhoz, hogy a Qdrantban releváns rekordokat kereshessünk.

---

## 5. Legrelevánsabb rekordok lekérése Qdrantból

- A generált embedding alapján a Qdrant **Cosine similarity** használatával visszaadja a legrelevánsabb rekordokat.
- Például a **top 5 találat** kerül kiválasztásra a kérdéshez legközelebb álló vektorok alapján.

---

## 6. Kontextus előkészítése a GPT-5 számára

- A lekért rekordokból szöveges kontextust készítünk: minden rekord mezőit összefűzzük, pl. `Eszköz: 1000113 | Telephely: nikszf4421`.
- A kontextus lesz a GPT-5 **prompt része**, amely alapján a modell összefüggő, természetes nyelvű választ ad.

---

## 7. GPT-5 hívás és válasz generálás

- A GPT-5 chat API-ját hívjuk meg, ahol:
  - `system` szerep: "Segítséget nyújtok inventory kérdésekre."
  - `user` szerep: felhasználói kérdés + releváns kontextus a Qdrantból.
- A GPT-5 feldolgozza az információkat és **érthető, természetes nyelvű választ generál** a kérdésre.

---

## 8. Kérdés és AI válasz kiírása

- A felhasználói kérdés és a GPT-5 válasza kiírásra kerül a notebookban vagy konzolon.
- Ez lehetővé teszi, hogy **áttekinthető legyen a kérdés-válasz pár**, és ellenőrizhetjük a rendszer pontosságát.


---

Ez a pipeline biztosítja, hogy a felhasználó **egy kérdésre kapjon konkrét, releváns választ**, az inventory adatbázis alapján.
